In [4]:
import os
import json
from datetime import datetime
from typing import List, Dict

import requests

API_KEY = os.getenv("YELP_API_KEY")
if not API_KEY:
    raise EnvironmentError("Missing YELP_API_KEY env var")

In [5]:
LATITUDE  = 32.8801                         # UC-San Diego (Geisel Library)
LONGITUDE = -117.2340
RADIUS_M  = 4000     
HEADERS = {"Authorization": f"Bearer {API_KEY}"}
SEARCH_URL = "https://api.yelp.com/v3/businesses/search"
DETAIL_URL = "https://api.yelp.com/v3/businesses/{}"
REVIEWS_URL = "https://api.yelp.com/v3/businesses/{}/reviews"

In [10]:
def _close_str(business: Dict) -> str:
    """Return 'Open until 9:00 PM'-style string for today, or '' if unknown."""
    hours = business.get("hours")
    if not hours:
        return ""
    today = datetime.now().weekday()  # 0 = Mon
    period = next(
        (p for p in hours[0]["open"] if p["day"] == today), None
    )
    if not period:
        return ""
    close = period["end"]  # e.g. '2130'
    hh, mm = int(close[:2]), close[2:]
    suffix = "PM" if hh >= 12 else "AM"
    hh = hh if 1 <= hh <= 12 else abs(hh - 12)
    return f"Open until {hh}:{mm} {suffix}"


def fetch_restaurants(limit: int = 500) -> List[Dict]:
    params = {
        "latitude":  LATITUDE,
        "longitude": LONGITUDE,
        "radius":    RADIUS_M,     
        "categories": "restaurants",
        "limit": limit,
        "sort_by": "rating",
    }
    resp = requests.get(SEARCH_URL, headers=HEADERS, params=params)
    resp.raise_for_status()
    return resp.json().get("businesses", [])


def build_entry(biz: Dict, idx: int) -> Dict:
    # Enrich with business detail & reviews
    bid = biz["id"]
    detail = requests.get(DETAIL_URL.format(bid), headers=HEADERS).json()
    reviews = requests.get(REVIEWS_URL.format(bid), headers=HEADERS).json().get("reviews", [])

    # Helper values
    miles = round(biz.get("distance", 0) / 1609.34, 1)  # meters → miles
    cuisine = detail["categories"][0]["title"] if detail.get("categories") else ""
    neigh = detail["location"].get("neighborhoods") or []
    location = neigh[0] if neigh else detail["location"].get("city", "San Diego")

    entry = {
        "id": idx,
        "name": detail["name"],
        "cuisine": cuisine,
        "price": detail.get("price", ""),
        "distance": miles,
        "rating": detail["rating"],
        "location": location,
        "phone": detail.get("display_phone", ""),
        "hours": _close_str(detail),
        "image": detail.get("image_url", ""),
        "menuImages": detail.get("photos", [])[:3],
        "reviews": [
            {"text": r["text"], "author": r["user"]["name"]} for r in reviews
        ][:6],  # Yelp returns up to 3; truncate/pad below
    }

    # Pad to exactly 6 review slots if necessary
    while len(entry["reviews"]) < 6:
        entry["reviews"].append({"text": "", "author": ""})

    return entry


In [12]:
raw = fetch_restaurants()


HTTPError: 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?latitude=32.8801&longitude=-117.234&radius=4000&categories=restaurants&limit=500&sort_by=rating

In [8]:
structured = [build_entry(b, i + 1) for i, b in enumerate(raw)]

In [9]:
with open("restaurants.json", "w", encoding="utf-8") as f:
    json.dump(structured, f, indent=2, ensure_ascii=False)